In [2]:
!pip install transformers

In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)

In [5]:
df = pd.read_csv("examples.csv")

In [ ]:
df

,sentence,question,set_id,type,answer,correct_option,incorrect_option
0,The child that the neighbor that the guest scared liked fell.,Who did the guest scare?,s1,second_center,The neighbor,neighbor,child
1,The child that the neighbor that the guest scared liked fell.,Who did the neighbor like?,s1,first_center,The child,child,guest
2,"The child, who was liked by the neighbor scared by the guest, fell.",Who did the guest scare?,s1,second_center_nd,The neighbor,neighbor,child
3,"The child, who was liked by the neighbor scared by the guest, fell.",Who did the neighbor like?,s1,first_center_nd,The child,child,guest
4,The man that the investigator that the journalist admired checked laughed.,Who did the journalist admire?,s2,second_center,The investigator,investigator,man
5,The man that the investigator that the journalist admired checked laughed.,Who did the investigator check?,s2,first_center,The man,man,journalist
6,"The man, who was checked by the investigator admired by the journalist, laughed.",Who did the journalist admire?,s2,second_center_nd,The investigator,investigator,man
7,"The man, who was checked by the investigator admired by the journalist, laughed.",Who did the investigator check?,s2,first_center_nd,The man,man,journalist
8,The thief that the mover that the architect knew surprised got caught.,Who did the architect know?,s3,second_center,The mover,mover,thief
9,The thief that the mover that the architect knew surprised got caught.,Who did the mover surprise?,s3,first_center,The thief,thief,architect


In [ ]:
!pip install fastchat pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 kB 11.7 MB/s eta 0:00:00


In [6]:
MODEL_NANE = "lmsys/vicuna-7b-v1.3"
DEVICE = torch.device('cuda' if torch.cuda.is_available() else "cpu")

In [28]:
def generate_prompt(sentence, question):
    prompt = (
    f"Sentence: The capital of France is one of the most visited cities in the world.\n"
    f"Question: What is the capital of France?\n"
    f"Answer: Paris\n\n"
    f"Sentence: Water is composed of two hydrogen atoms and one oxygen atom.\n"
    f"Question: What is the chemical symbol for water?\n"
    f"Answer: H₂O\n\n"
    f"Sentence: The Earth is divided into large landmasses known as continents.\n"
    f"Question: How many continents are there on Earth?\n"
    f"Answer: Seven\n"
    f"Sentence: {sentence}\n"
    f"Question: {question}\n"
    f"Answer: The")

    return prompt

In [44]:
def predict_answer(model, tokenizer, sentence, correct_option, incorrect_option, question, num_tokens=5):
    prompt = generate_prompt(sentence, question)
    input_ids_sentence = tokenizer(prompt, return_tensors="pt").input_ids.to(DEVICE)
    correct_tokens = tokenizer(correct_option, return_tensors="pt").input_ids.squeeze()
    incorrect_tokens = tokenizer(incorrect_option, return_tensors="pt").input_ids.squeeze()

    # Get the possible token IDs (correct and incorrect tokens)
    correct_token_ids = correct_tokens.tolist()
    incorrect_token_ids = incorrect_tokens.tolist()

    # Initialize the predicted tokens list with the sentence input
    generated_ids = input_ids_sentence
    answer_ids = tokenizer.encode("The")

    # here we are generting sequence in num_tokens length, to handle multiple tokens answers
    # in the accuracy calculation we will ceck only the first token of the answer
    for _ in range(num_tokens):
        with torch.no_grad():
            outputs = model(generated_ids).logits

        next_token_logits = outputs[:, -1, :]

        # Extract logits only for the tokens in correct and incorrect options
        valid_token_ids = correct_token_ids + incorrect_token_ids
        next_token_logits_valid = next_token_logits[:, valid_token_ids]

        # Find the argmax of the valid token logits
        predicted_token_id = valid_token_ids[torch.argmax(next_token_logits_valid)]

        predicted_token_id_tensor = torch.tensor([[predicted_token_id]], device=generated_ids.device)
        answer_ids.append(predicted_token_id)
        generated_ids = torch.cat((generated_ids, predicted_token_id_tensor), dim=-1)

    predicted_text = tokenizer.decode(answer_ids, skip_special_tokens=True)
    return predicted_text

In [8]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NANE)
if tokenizer.pad_token_id is None:
   tokenizer.pad_token_id = tokenizer.eos_token_id

model = AutoModelForCausalLM.from_pretrained(MODEL_NANE, torch_dtype=torch.float16)
model.to(DEVICE)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you l

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      

In [43]:
correct_predictions = 0
total_predictions = 0

for index, row in df.iterrows():
    sentence = row['sentence']
    correct_option = row['correct_option']
    incorrect_option = row['incorrect_option']
    true_answer = row['answer']
    question = row['question']

    predicted_answer = predict_answer(model, tokenizer, sentence, correct_option, incorrect_option, question)
    print(f"Sentence: {sentence}")
    print(f"Correct Option: {correct_option}")
    print(f"Incorrect Option: {incorrect_option}")
    print(f"True Answer: {true_answer}")
    print(f"Predicted Answer: {predicted_answer}")

    if predicted_answer.startswith(true_answer):
        correct_predictions += 1
        print("Correct prediction!")
    else:
        print("Incorrect prediction.")

    print("-" * 50)

    total_predictions += 1

# Calculate accuracy
accuracy = correct_predictions / total_predictions * 100
print(f"Accuracy: {accuracy:.2f}%")

Sentence: The child that the neighbor that the guest scared liked fell.
Correct Option: neighbor
Incorrect Option: child
True Answer: The neighbor
Predicted Answer: The child neighbor child child neighbor
Incorrect prediction.
--------------------------------------------------
Sentence: The child that the neighbor that the guest scared liked fell.
Correct Option: child
Incorrect Option: guest
True Answer: The child
Predicted Answer: The child guest child guest child
Correct prediction!
--------------------------------------------------
Sentence: The child, who was liked by the neighbor scared by the guest, fell.
Correct Option: neighbor
Incorrect Option: child
True Answer: The neighbor
Predicted Answer: The child child child child child
Incorrect prediction.
--------------------------------------------------
Sentence: The child, who was liked by the neighbor scared by the guest, fell.
Correct Option: child
Incorrect Option: guest
True Answer: The child
Predicted Answer: The child child